In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import  MinMaxScaler
from sklearn.metrics import classification_report

#Lê o arquivo
df = pd.read_csv('banco.tsv', '\t')